In [103]:
import json
import numpy as np
import requests
from tqdm import tqdm

url = "http://127.0.0.1:8000/"
# /predict POST -> all 30 attributes
# /conversion POST -> user_id, value
# /resultsAB GET -> get all conversions in JSON
# /resetAB POST -> reset memory of conversions

headers = {
    'Content-Type': 'application/json'
}

In [104]:
reset_response = requests.request("POST", url+'resetAB/')


In [105]:
N = 1000
print("Processing ",N," events...")

Processing  1000  events...


In [106]:
for i in tqdm(range(N)):
    # generator of attributes
    attributes = dict()
    attributes['user_id'] = i
    attributes['price'] = round(np.random.normal(2000, 1000),0)
    attributes['product_views_count'] = int(abs(round(np.random.normal(0, 15), 0)))
    attributes['category_view_count'] = int(abs(round(np.random.normal(0, 11), 0)))
    attributes['bought_with_lower_count'] = int(abs(round(np.random.normal(0, 6), 0)))
    attributes['bought_category_count'] = int(abs(round(np.random.normal(0, 7), 0)))
    attributes['time'] = abs(round(np.random.normal(0, 1000), 2))
    city_index = int(round(np.random.uniform(0, 7), 0))
    for j in range(8):
        attributes['one_hot_city' + str(j)] = 1 if j == city_index else 0

    category_index = round(np.random.uniform(0, 14))
    for j in range(15):
        attributes['one_hot_category' + str(j)] = 1 if j == category_index else 0

    # send request for predictions of discount
    json_data = json.dumps(attributes, ensure_ascii=True)
    response = requests.request("POST", url+'predict/', headers=headers, data=json_data)

    #%%

    discount = response.json()['discount']

    #%%

    # generate decision about buying
    import random
    rand_num = random.random()
    probability = (discount/100.0 + 0.1)
    buying = (rand_num < probability)
    # print("rand_num: ", rand_num)
    # print()
    # print("Decision ", i, " = ",buying)


    # if buying then post conversion to server
    if not buying:
        continue
    conversion_dictionary = dict()
    conversion_dictionary['user_id'] = i
    conversion_dictionary['value'] = float(attributes['price']*(40-discount)/100)
    conversion_data = json.dumps(conversion_dictionary)
    conversion_response = requests.request("POST", url+'conversion/', headers=headers, data=conversion_data)
    # print(conversion_response.content)

100%|██████████| 1000/1000 [00:22<00:00, 43.48it/s]


In [107]:
# after all actions

# get all conversions in JSON
all_conversions = requests.request("GET", url+'resultsAB/')
conversions = all_conversions.json()

# count conversions and print results
value0 = 0
value1 = 0

count0 = 0
count1 = 0
for conversion in conversions:
    if int(conversion['model_id']) == 0:
        value0 += float(conversion['value'])
        count0 += 1
    else:
        value1 += float(conversion['value'])
        count1 += 1
print("Dummy conversion number: ", count0)
print("Tuned conversion number: ", count1)

print('\n')

print("Dummy conversion value: ", round(value0, 2))
print("Tuned conversion value: ", round(value1, 2))

Dummy conversion number:  74
Tuned conversion number:  142


Dummy conversion value:  47501.5
Tuned conversion value:  85312.5
